# 검색 알고리즘 실험
- 특정 문서를 입력받아 코사인 유사도를 기반으로 유사한 문서를 반환하는 실험을 진행해보자.

1. TF-IDF + NMF
1. Word2vec

## Import

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import Normalizer
from konlpy.tag import Mecab

## Data
>sourced by momtalk(https://www.momtalk.kr/talk/list/all)

1. `df1`: 육아정보 포스팅
1. `df2`: 유아용품 포스팅
1. `df` : 위의 두 데이터를 합친 것

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')

df = pd.concat([df1.text, df2.text, df3.text, df4.text, 
                df5.text, df6.text, df7.text, df8.text])
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
17301,아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...
17302,세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...
17303,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...
17304,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...
17305,소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...


## TF-IDF

In [3]:
def tokenize(text):
    mecab = Mecab()
    tokens = mecab.pos(text)
    key_tokens = [token[0] for token in tokens 
                 if ('NNG' in token or 'NNP' in token or 'VA' in token or 'VV' in token)]
    key_tokens = ' '.join(key_tokens)
    return key_tokens

    
df['token'] = df.text.apply(tokenize)

In [4]:
df.tail()

,text,token
17301,아이는 현재 신체와 정신이 모두 발달 중에 있다 신체 발달은 근육의 발달을 정신 발...,아이 신체 정신 발달 있 신체 발달 근육 발달 정신 발달 지적 호기심 발달 포함 공...
17302,세 살 적 버릇이 여든까지 간다는 말이 있다 밥상머리 예절도 예외는 아니다 아이에게...,버릇 말 있 밥상머리 예절 예외 아이 식습관 가족 밥상 앞 앉 우선 가족 자신 자리...
17303,아이의 키가 자라고 몸무게가 늘어나면서 아이 몸을 구성하고 있는 장기들도 커지고 무...,아이 키 자라 몸무게 늘어나 아이 몸 구성 장기 무게 성장 발달 신체 부분 일정 순...
17304,눈치는 직관적이고 비언어적인 방법으로 의사소통을 하고 문제를 해결하는 놀라운 능력이...,눈치 직관 언어 방법 의사소통 하 문제 해결 능력 사람 관계 맺 사람 생각 감정 이...
17305,소리를 들으면 사람은 그 소리를 그대로 흉내 낼 수 있다 이것이 청력의 힘이다 말소...,소리 사람 소리 흉내 있 청력 힘 말 소리 아이 듣 소리 모방 소리 소리 감정 느낌...


In [5]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['token'])

print(vectors.shape)

(17306, 52618)


## NMF(Non-negative Matrix Factorization)
- sparse coding처럼 dictionary와 encoded vector 학습
- parameter가 0보다 큰 특성으로 feature matrix가 원본 데이터들과 어떠한 관계를 가지는지 해석이 가능
- 연산에 약간의 시간 소요

In [7]:
%%time
vector_array = vectors.toarray()
nmf = NMF(n_components=40)
nmf.fit(vector_array)
features = nmf.transform(vector_array)

CPU times: user 10min 1s, sys: 8.73 s, total: 10min 9s
Wall time: 1min 22s


## Normalize extracted feature set
- 0과 1사이의 값으로 정규화하여 feature vector의 길이를 1로 고정

In [8]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

print(norm_features[0:2])

[[0.67352631 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.00443486 0.         0.00631638 0.         0.5910503
  0.21863979 0.32410772 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.14749596 0.         0.1495261
  0.         0.         0.         0.        ]
 [0.47687154 0.         0.0504711  0.08334657 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.83785974 0.0655659  0.         0.         0.01580153 0.14878155
  0.         0.         0.14105379 0.         0.         0.02364115
  0.         0.         0.         0.08507854 0.         0.
  0.         0.         0.         0.08193072]]


In [9]:
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
17301,0.480891,0.0,0.000000,0.021793,0.017334,0.000000,0.0,0.000000,0.000000,0.063648,...,0.000000,0.079535,0.046299,0.000000,0.000000,0.0,0.024815,0.0,0.019328,0.076318
17302,0.633368,0.0,0.234742,0.032812,0.000000,0.000000,0.0,0.000000,0.000000,0.443117,...,0.010047,0.000000,0.000000,0.131166,0.000000,0.0,0.127371,0.0,0.000000,0.003848
17303,0.140065,0.0,0.000000,0.025828,0.000000,0.000000,0.0,0.007885,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.020018,0.037004,0.0,0.000000,0.0,0.000000,0.165281
17304,0.355064,0.0,0.040009,0.146795,0.005780,0.000000,0.0,0.018891,0.017241,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.009785,0.0,0.000000,0.0,0.000000,0.000000
17305,0.019328,0.0,0.000000,0.035861,0.000000,0.128988,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.043088


`df`의 맨 첫번째 텍스트를 정규화 한 40차원 벡터의 값

In [10]:
print(df.loc[0].text)
article = df_features.loc[0]
len(article)

6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요  매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다 


40

In [11]:
similarities=df_features.dot(article)
top=similarities.nlargest(10)

texts = df.loc[top.index]['text'].tolist()
i = 0
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    #print(text+'\n')
    i = i+1

TITLE :0 Similarities:0.9999999999999999
TITLE :1788 Similarities:0.9477425746130276
TITLE :869 Similarities:0.9346397571991394
TITLE :11623 Similarities:0.910524719933069
TITLE :5811 Similarities:0.8944208085441908
TITLE :4592 Similarities:0.8942557643569937
TITLE :7374 Similarities:0.8936376058076692
TITLE :10384 Similarities:0.8878380735030909
TITLE :1801 Similarities:0.8872853819809849
TITLE :9830 Similarities:0.8870334654496674


In [12]:
i = 0
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

TITLE :0 Similarities:0.9999999999999999
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요  매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다 

TITLE :1788 Similarities:0.9477425746130276
 아이가 갑자기 학원을 안 가겠대요 이미지 아이아트그만둬도 될까 아이가 갑자기 학원을 안 가겠대요 아이가 너무

---

## Word2Vec
- 위의 TF-IDF 임베딩이 아닌 Word2Vec 임베딩으로 실험해보자.
- 음수의 값이 포함되서는 안되는 NMF는 활용할 수 없다. 어차피 sparse하지 않으면서 TF-IDF에 비해 훨씬 저차원인 성격을 지닌 word2vec 임베딩은 바로 유사도를 계산할 수 있어 더욱 편리하게 활용할 수 있을 것이라 판단된다.

In [13]:
from gensim.models import Word2Vec
from konlpy.tag import Mecab
import pandas as pd
import numpy as np

In [14]:
model = Word2Vec.load('data/tokmom_word2vec_only_noun')
df = pd.read_csv('data/text_for_word2vec.csv')
df = df.drop(index=df[df['text'] == ""].index)
df = df.reset_index(drop=True)

In [15]:
%%time
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)

CPU times: user 41.9 s, sys: 21.4 s, total: 1min 3s
Wall time: 1min 3s


In [18]:
def make_to_vector(nouns):
    mean_of_noun_vectors = []
    except_tokens = []
    for noun in nouns:
        try:
            noun_vector = model.wv.get_vector(noun)
            mean_of_noun_vectors.append(noun_vector)
        except KeyError:
#             print("{} is not exist in the vocab".format(noun))
            except_tokens.append(noun)
            continue
    try:
        mean_of_noun_vectors = sum(mean_of_noun_vectors) / len(mean_of_noun_vectors)
        return mean_of_noun_vectors
    except ZeroDivisionError:
        return np.nan

In [19]:
df['text_vector'] = df['noun'].apply(make_to_vector)
df = df.dropna()
df = df.reset_index(drop=True)

In [20]:
df['text_vector']

0        [0.087730765, 0.048226427, 0.13503045, 0.07771...
1        [-0.030752694, 0.10367098, 0.20024624, 0.03413...
2        [-0.0049099764, 0.14312822, 0.2286539, 0.05528...
3        [-0.031288117, 0.09118678, 0.20046811, 0.04638...
4        [-0.02507854, 0.040282574, 0.19808322, 0.00356...
                               ...                        
17326    [0.053981233, 0.06268845, 0.068942174, -0.0503...
17327    [0.10072067, 0.06669665, 0.17370501, 0.0337225...
17328    [-0.08723935, 0.09772762, 0.12012127, -0.06258...
17329    [-0.038680922, 0.0562258, 0.17001307, -0.02443...
17330    [-0.13421059, 0.041578338, 0.14118925, -0.1181...
Name: text_vector, Length: 17331, dtype: object

In [21]:
vectors = np.zeros(100,)
for vector in df.text_vector:
    vectors = np.vstack([vectors, vector])
vectors = vectors[1:, :]
vectors

array([[ 0.08773077,  0.04822643,  0.13503045, ..., -0.18616493,
        -0.08267397, -0.28164151],
       [-0.03075269,  0.10367098,  0.20024624, ..., -0.21707731,
        -0.10976584, -0.32339045],
       [-0.00490998,  0.14312822,  0.22865389, ..., -0.15026751,
        -0.1224198 , -0.28446805],
       ...,
       [-0.08723935,  0.09772762,  0.12012127, ..., -0.24863696,
        -0.07963976, -0.34973449],
       [-0.03868092,  0.0562258 ,  0.17001307, ..., -0.15213731,
        -0.03560679, -0.35873312],
       [-0.13421059,  0.04157834,  0.14118925, ..., -0.26515555,
        -0.02389128, -0.29719433]])

>ndarray 형식으로 예쁘게 만들어야 되는데 이런 식으로 밖에 못만드는게 내 실력 미달인듯... ㅠ

## Normalize extracted feature set
- 유사도 계산의 편의를 위해 0과 1사이의 값으로 정규화

In [22]:
from sklearn.preprocessing import Normalizer

normalizer = Normalizer()
norm_features=normalizer.fit_transform(vectors)
df_features = pd.DataFrame(norm_features,index=df.index.tolist())
df_features.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
17326,0.030373,0.035272,0.038790,-0.028332,-0.043771,0.035967,0.123041,-0.097430,-0.060558,0.018628,...,-0.130157,0.000489,-0.058270,-0.048787,-0.066302,-0.137358,0.009278,-0.176554,-0.048011,-0.137570
17327,0.054087,0.035816,0.093280,0.018109,-0.065598,0.023841,0.155532,-0.067007,0.020559,0.038778,...,-0.090866,0.031684,-0.030941,-0.088677,-0.015559,-0.134087,-0.047499,-0.133159,-0.021583,-0.156367
17328,-0.046416,0.051996,0.063911,-0.033296,-0.074033,0.043958,0.142312,-0.106207,0.006592,0.008481,...,-0.024731,0.071680,0.050831,-0.013487,-0.043440,-0.103528,-0.016552,-0.132288,-0.042373,-0.186078
17329,-0.020391,0.029641,0.089626,-0.012879,-0.129142,0.042660,0.154226,-0.125154,-0.012686,-0.014520,...,-0.045848,0.038300,0.037750,-0.024383,0.005873,-0.090043,-0.050505,-0.080202,-0.018771,-0.189114
17330,-0.070450,0.021825,0.074113,-0.062016,-0.055207,0.026042,0.143592,-0.039177,-0.006960,-0.049476,...,-0.033113,0.093419,0.030329,0.028339,-0.078844,-0.101108,-0.035709,-0.139185,-0.012541,-0.156003


In [23]:
article = df_features.loc[0]
similarities=df_features.dot(article) # cosine similarity
top=similarities.nlargest(5)

texts = df.loc[top.index]['text'].tolist()

i = 0
print("<비교할 문서>")
for text in texts:
    df_idx = top.index[i]
    print('TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
    print(text+'\n')
    i = i+1

<비교할 문서>
TITLE :0 Similarities:1.0
6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요  매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다 

TITLE :15619 Similarities:0.9726340202919597
우리 아이의 첫 보육시설은 큰 의미를 갖는다 가정을 벗어나 더 큰 사회적 집단 속에 첫발을 딛는 순간이다 더 이상 부모

### 가상의 키워드를 입력하여 유사도 계산해보기
- 이번에는 챗봇처럼 짧은 문장(아마 키워드 위주)을 입력받아 유사도 계산을 해보자.
- word2vec vocabulary에 없는 단어를 입력받을 경우 에러가 발생하는 문제가 있다.

In [ ]:
tokenizer = Mecab()

while True:
    input_text = input("Input text:")
    if input_text == "exit":
        print("Bye")
        break
        
    tokens = tokenizer.morphs(input_text)

    try:
        input_vector = [model.wv.get_vector(token) for token in tokens]
        input_vector = sum(input_vector) / len(input_vector)

        similarities=df_features.dot(input_vector)
        top=similarities.nlargest(5)

        texts = df.loc[top.index]['text'].tolist()

        i = 0
        for text in texts:
            df_idx = top.index[i]
            print('\n'+'TITLE :'+str(df_idx)+" Similarities:"+ str(top[df_idx]))
            print(text+'\n')
            i = i+1
            
    except:
        print("'{}'은(는) 제가 아직 잘 이해하지 못하겠어요ㅠㅠ ".format(input_text))
    

Input text:중고 유모차 상품

TITLE :2322 Similarities:2.3575462087406276
 절충형 유모차 2018 신상 리스트 이미지 출처 페도라 따끈한 신상이 왔어요 절충형 유모차 2018 신상 리스트 유모차는 다양한 종류가 있는데요 그중에서도 디럭스형의 편안함과 휴대형의 실용성을 갖춘 절충형 유모차는 꾸준히 엄마들의 사랑을 받고 있어요 2018년 출시 및 리뉴얼된 다양한 절충형 유모차를 서 소개합니다 페도라 2018 L3 편리함과 실용성을 모두 갖춘 양대면 컴팩트 절충형 유모차 페도라 L3의 2018 신상 유모차예요 7 3kg의 무게에 5단 캐노피와 선셰이드로 햇빛을 가려주고 편리한 시트 각도 조절과 넓은 통풍창으로 쾌적함을 선사해요 그밖에도 2018버전은 다양하게 업그레이드 됐답니다  제품  


TITLE :10438 Similarities:2.341038070645973
28일부터 10월 7일까지 진행되는 코리아세일페스타에 페도라 유모차와 카시트가 참가한다 이는 유아용품 중 유모차와 카시트라는 발육기 카테고리 중 최초로 참가이며 대한민국 대표 쇼핑 관광 축제에 참가하는 것에 의의가 있다대한민국 대표 유아용품 브랜드 페도라는 유모차와 카시트를 유통 및 판매하고 있으며 공식 온라인 쇼핑몰인 쁘레베베샵에서 최고 56 할인 판매를 진행한다 유모차로는 양대면이 가능한 절충휴대용 유모차인 페도라 L3 올라운드 카시트로 7세까지 사용 가능한 C3 9개월부터 12세까지 사용 가능한 M5나 C2 카시트에 이르기까지 다양한 품목에 대해 할인 행사를 진행하고 있다또한 유아식탁의자 P3도 행사 진행 품목에 포함돼 있으며 자전거유모차에는 요가와 러츠 제품을 최고 56까지 할인 판매를 진행하고 있다페도라 인기상품을 한 자리에서 볼 수 있는 쁘레베베샵은 페도라 유모차 카시트의 공식 온라인 쇼핑몰로 최근 리뉴얼해 네이버 톡톡 문의와 카카오톡 플러스 친구 등의 채널을 통해 고객과의 커뮤니케이션을 활성화하고 인스타그램 등의 SNS를 통한 다양한 이벤트로 